In [ ]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks


Обработка естественного языка с нуля: перевод с помощью сети последовательностей и внимания
===============================================================================


В этом проекте мы будем обучать нейронную сеть переводить с русского на английский.

\... в разной степени успешности.

Это становится возможным благодаря простой, но мощной идее [сети последовательности к последовательности](https://arxiv.org/abs/1409.3215), в которой две рекуррентные нейронные сети работают вместе, чтобы преобразовать одну последовательность в другую. Сеть энкодера сжимает входную последовательность в вектор, а сеть декодера разворачивает этот вектор в новую последовательность.

![](https://pytorch.org/tutorials/_static/img/seq-seq-images/seq2seq.png)

Чтобы улучшить эту модель, мы будем использовать [механизм внимания](https://arxiv.org/abs/1409.0473), который позволяет декодеру научиться фокусироваться на определенном диапазоне входной последовательности.

**Требования**

In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Загрузка файлов данных
==================

Для этого проекта данные представляют собой набор множеств тысяч пар английских и русских переводов.

Пары английский-французский слишком большие, чтобы включить их в репозиторий, поэтому загрузите их в `data/rus-eng.txt`, прежде чем продолжить. Файл представляет собой список пар переводов, разделенных табуляцией:

``` {.sourceCode .sh}
Wow!	Здорово!
```

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>ПРИМЕЧАНИЕ:</strong></div>
<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">
<p>Загрузите данные с <a href="https://www.manythings.org/anki/rus-eng.zip">этого</a> сайта и извлеките их в текущий каталог.</p>
</div>


Similar to the character encoding used in the character-level RNN
tutorials, we will be representing each word in a language as a one-hot
vector, or giant vector of zeros except for a single one (at the index
of the word). Compared to the dozens of characters that might exist in a
language, there are many many more words, so the encoding vector is much
larger. We will however cheat a bit and trim the data to only use a few
thousand words per language.

![](https://pytorch.org/tutorials/_static/img/seq-seq-images/word-encoding.png)


Нам понадобится уникальный индекс для каждого слова, чтобы использовать его в качестве входных и целевых данных для сетей позже. Чтобы отслеживать все это, мы будем использовать вспомогательный класс под названием `Lang`, который содержит словари слово → индекс (`word2index`) и индекс → слово (`index2word`), а также подсчет каждого слова `word2count`, который будет использоваться позже для замены редких слов.


In [ ]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

Все файлы находятся в формате Unicode. Для упрощения мы преобразуем символы Unicode в ASCII, приведем все к нижнему регистру и удалим большую часть знаков пунктуации.


In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^а-яА-Яa-zA-Z!?]+", r" ", s)
    return s.strip()

Чтобы прочитать файл данных, мы разобьем файл на строки, а затем разобьем строки на пары. Все файлы имеют формат Английский → Другой язык, поэтому если мы хотим перевести с Другого языка → Английский, я добавил флаг `reverse`, чтобы перевернуть пары.


In [ ]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

Поскольку у нас очень *много* примеров предложений, и мы хотим быстро обучить что-то, мы сократим набор данных до относительно коротких и простых предложений. Максимальная длина здесь составляет 10 слов (включая конечную пунктуацию), и мы фильтруем предложения так, чтобы они переводились в форму "Я" или "Он" и т.д. (учитывая апострофы, замененные ранее).


In [ ]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

Полный процесс подготовки данных включает в себя следующее:

- Чтение текстового файла и разбиение на строки, разбиение строк на пары
- Нормализация текста, фильтрация по длине и содержанию
- Создание списков слов из предложений в парах

In [ ]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('rus', 'eng', True)
print(random.choice(pairs))

Reading lines...
Read 487600 sentence pairs
Trimmed to 30189 sentence pairs
Counting words...
Counted words:
eng 10444
rus 4333
['ты выше отца', 'you re taller than your father']


Модель Seq2Seq
=================

Рекуррентная нейронная сеть, или RNN, - это сеть, которая оперирует над последовательностью и использует свой собственный вывод в качестве входа для последующих шагов.

Сеть [Последовательность к Последовательности](https://arxiv.org/abs/1409.3215), или seq2seq, или [Сеть Энкодер-Декодер](https://arxiv.org/pdf/1406.1078v3.pdf), - это модель, состоящая из двух RNN, называемых энкодером и декодером. Энкодер считывает входную последовательность и выводит один вектор, а декодер читает этот вектор, чтобы произвести выходную последовательность.

![](https://pytorch.org/tutorials/_static/img/seq-seq-images/seq2seq.png)

В отличие от прогнозирования последовательности с помощью одной RNN, где каждый вход соответствует выходу, модель seq2seq освобождает нас от длины и порядка последовательности, что делает ее идеальной для перевода между двумя языками.

Рассмотрим предложение `Je ne suis pas le chat noir` → `I am not the black cat`. Большинство слов во входном предложении имеют прямой перевод в выходном предложении, но они немного отличаются по порядку, например, `chat noir` и `black cat`. Из-за конструкции `ne/pas` во входном предложении также есть еще одно слово. Было бы сложно непосредственно из последовательности входных слов получить правильный перевод.

С помощью модели seq2seq энкодер создает один вектор, который, в идеальном случае, кодирует "смысл" входной последовательности в один вектор - одну точку в некотором N-мерном пространстве предложений.

Энкодер
===========

Энкодер сети seq2seq - это рекуррентная нейронная сеть (RNN), которая выдает некоторое значение для каждого слова из входного предложения. Для каждого входного слова энкодер выдает вектор и скрытое состояние, и использует скрытое состояние для следующего входного слова.

![](https://pytorch.org/tutorials/_static/img/seq-seq-images/encoder-network.png)

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

Декодер
===========

Декодер - это еще одна рекуррентная нейронная сеть (RNN), которая принимает вектор(ы) выхода энкодера и выдает последовательность слов для создания перевода.

Простой декодер
==============

В самом простом декодере seq2seq мы используем только последний выход энкодера. Этот последний выход иногда называется *вектором контекста*, так как он кодирует контекст из всей последовательности. Этот вектор контекста используется в качестве начального скрытого состояния декодера.

На каждом шаге декодирования декодеру передается токен ввода и скрытое состояние. Начальный входной токен - это токен начала строки `<SOS>`, а первое скрытое состояние - это вектор контекста (последнее скрытое состояние энкодера).

![](https://pytorch.org/tutorials/_static/img/seq-seq-images/decoder-network.png)

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden, None # We return `None` for consistency in the training loop

    def forward_step(self, input, hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden

# Декодер внимания

Если между энкодером и декодером передается только вектор контекста, то этот единственный вектор несет на себе бремя кодирования всего предложения.

Механизм внимания позволяет сети декодера "фокусироваться" на разных частях выходов энкодера для каждого шага собственных выходов декодера. Сначала мы вычисляем набор *весов внимания*. Они будут умножены на векторы выхода энкодера для создания взвешенной комбинации. Результат (называемый `attn_applied` в коде) должен содержать информацию о конкретной части входной последовательности и таким образом помогать декодеру выбирать правильные выходные слова.

![](https://i.imgur.com/1152PYf.png)

Вычисление весов внимания выполняется с помощью еще одного слоя прямого распространения `attn`, используя вход декодера и скрытое состояние в качестве входных данных. Поскольку в обучающих данных есть предложения всех размеров, чтобы фактически создать и обучить этот слой, нам нужно выбрать максимальную длину предложения (длину ввода для выходов энкодера), которую он может применять. Предложения максимальной длины будут использовать все веса внимания, в то время как более короткие предложения будут использовать только первые несколько.

![](https://pytorch.org/tutorials/_static/img/seq-seq-images/attention-decoder-network.png)

Механизм внимания Бахданау, также известный как аддитивное внимание, является широко используемым механизмом внимания в моделях последовательность-последовательность, особенно в задачах нейронного машинного перевода. Он был представлен Бахданау и др. в своей статье под названием [Нейронный машинный перевод совместным обучением выравнивания и перевода](https://arxiv.org/pdf/1409.0473.pdf). Этот механизм внимания использует обучаемую модель выравнивания для вычисления оценок внимания между скрытыми состояниями энкодера и декодера. Он использует нейронную сеть прямого распространения для вычисления оценок выравнивания.

Однако существуют альтернативные механизмы внимания, такие как внимание Луонга, которое вычисляет оценки внимания, умножая скалярно скрытое состояние декодера на скрытые состояния энкодера. Оно не включает нелинейное преобразование, используемое в механизме внимания Бахданау.

In [ ]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

Обучение
========

Подготовка обучающих данных
-----------------------

Для обучения нам потребуется тензор входных данных (индексы слов во входном предложении) и целевой тензор (индексы слов в целевом предложении) для каждой пары. При создании этих векторов мы добавим токен EOS в обе последовательности.

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size):
    input_lang, output_lang, pairs = prepareData('rus', 'eng', True)

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return input_lang, output_lang, train_dataloader

Обучение модели
==================

Для обучения мы передаем входное предложение через энкодер и отслеживаем каждый выход и последнее скрытое состояние. Затем декодеру передается токен `<SOS>` в качестве его первого входа, а последнее скрытое состояние энкодера - в качестве его первого скрытого состояния.

"Принуждение учителя" - это концепция использования реальных целевых выходных данных в качестве следующего входа, вместо использования догадки декодера в качестве следующего входа. Использование принуждения учителя позволяет сети быстрее сходиться, но [когда обученная сеть используется, она может проявлять нестабильность](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.378.4095&rep=rep1&type=pdf).

Вы можете наблюдать выводы сетей с принуждением учителя, которые читают с логичной грамматикой, но удаляются далеко от правильного перевода - интуитивно она научилась представлять грамматику выхода и может "уловить" значение, когда учитель говорит ей первые несколько слов, но она не научилась правильно создавать предложение из перевода с самого начала.

Из-за свободы, которую дает нам автоград PyTorch, мы можем случайным образом решать, использовать ли принуждение учителя или нет с помощью простого оператора if. Увеличьте `teacher_forcing_ratio`, чтобы использовать его больше.

In [ ]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion):

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

Это вспомогательная функция для вывода затраченного времени и оценки оставшегося времени на основе текущего времени и процента выполнения.


In [ ]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

Весь процесс обучения выглядит следующим образом:

- Запустить таймер
- Инициализировать оптимизаторы и критерии
- Создать набор обучающих пар
- Начать с пустого массива потерь для построения графиков

Затем мы много раз вызываем функцию `train` и время от времени выводим прогресс (% примеров, затраченное время, оцененное время) и среднюю потерю.

In [ ]:
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

Визуализация результатов
================

Для построения графиков используется библиотека matplotlib, используя массив значений потерь `plot_losses`, сохраненных во время обучения.

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

Оценка
==========

Оценка в основном аналогична обучению, но здесь нет целевых данных, поэтому мы просто подаем предсказания декодера обратно ему самому на каждом шаге. Каждый раз, когда он предсказывает слово, мы добавляем его в выходную строку, и если он предсказывает токен EOS, мы останавливаемся. Мы также сохраняем выходы внимания декодера для отображения позже.

In [ ]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn

Мы можем оценить случайные предложения из обучающего набора и вывести вход, целевую и выходную информацию, чтобы сделать некоторые субъективные оценки качества:

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

Обучение и оценка
=======================

Со всеми этими вспомогательными функциями (которые кажутся лишней работой, но делают запуск нескольких экспериментов более простым) мы можем фактически инициализировать сеть и начать обучение.

In [ ]:
hidden_size = 128
batch_size = 32

input_lang, output_lang, train_dataloader = get_dataloader(batch_size)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

train(train_dataloader, encoder, decoder, 80, print_every=5, plot_every=5)

Reading lines...
Read 487600 sentence pairs
Trimmed to 30189 sentence pairs
Counting words...
Counted words:
eng 10444
rus 4333
10m 17s (- 154m 15s) (5 6%) 1.2607
20m 39s (- 144m 37s) (10 12%) 0.5007
30m 57s (- 134m 9s) (15 18%) 0.2826
41m 8s (- 123m 24s) (20 25%) 0.1894
51m 20s (- 112m 57s) (25 31%) 0.1427
61m 41s (- 102m 48s) (30 37%) 0.1167
71m 58s (- 92m 31s) (35 43%) 0.1009
82m 15s (- 82m 15s) (40 50%) 0.0904
92m 24s (- 71m 52s) (45 56%) 0.0824
102m 53s (- 61m 43s) (50 62%) 0.0776
113m 32s (- 51m 36s) (55 68%) 0.0726
124m 45s (- 41m 35s) (60 75%) 0.0691
135m 26s (- 31m 15s) (65 81%) 0.0666
145m 54s (- 20m 50s) (70 87%) 0.0645
156m 30s (- 10m 26s) (75 93%) 0.0621
167m 6s (- 0m 0s) (80 100%) 0.0609


Установите слои отсева в режим `eval`.

In [ ]:
encoder.eval()
decoder.eval()
evaluateRandomly(encoder, decoder)

> я невероятно занят
= i m incredibly busy
< i am incredibly busy <EOS>

> ты очень груб
= you re being very rude
< you re very rude <EOS>

> мы с томом почти ровесники
= i m almost as old as tom
< i expected almost about tom <EOS>

> я рад что мы договорились
= i m glad we agree
< i m glad we agree <EOS>

> он по праву гордится сыном
= he is justly proud of his son
< he is justly proud of his son <EOS>

> я готова идти за вами
= i m ready to follow you
< i am ready to follow you <EOS>

> я устал от такои жизни
= i m tired of living this kind of life
< i m tired of living this kind of life <EOS>

> он высок но его брат намного выше
= he is tall but his brother is much taller
< he is tall but his brother is much taller <EOS>

> я слишком устал для спора
= i m too tired to argue
< i m too tired to argue <EOS>

> мы работаем над этим прямо сеичас
= we re working on that right now
< we re working on that right now <EOS>



Визуализация внимания
=====================

Полезное свойство механизма внимания - его высокая интерпретируемость. Поскольку он используется для взвешивания конкретных выходов энкодера входной последовательности, мы можем представить себе, на что сеть сфокусирована больше всего на каждом временном шаге.

Для более удобного просмотра мы выполним дополнительную работу по добавлению осей и меток:

In [ ]:
def showAttention(input_sentence, output_words, attentions):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.cpu().numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(encoder, decoder, input_sentence, input_lang, output_lang)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions[0, :len(output_words), :])


evaluateAndShowAttention("я невероятно занят")

evaluateAndShowAttention('он по праву гордится сыном')

evaluateAndShowAttention('мы с томом почти ровесники')

evaluateAndShowAttention('мы работаем над этим прямо сеичас')

input = я невероятно занят
output = i am incredibly busy <EOS>
input = он по праву гордится сыном
output = he is justly proud of his son <EOS>
input = мы с томом почти ровесники
output = i expected almost about tom <EOS>


<ipython-input-27-6fa1f6ab916b>:8: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([''] + input_sentence.split(' ') +
<ipython-input-27-6fa1f6ab916b>:10: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels([''] + output_words)


input = мы работаем над этим прямо сеичас
output = we re working on that right now <EOS>


Выводы
=========


